In [44]:
# Import required libraries
import networkx as nx
import numpy as np
from scipy.special import factorial
from datetime import datetime
import logging

In [45]:
# Setup logging (analogous to R's sink to file)
logging.basicConfig(filename='experiment_1.log', 
                   level=logging.INFO,
                   format='%(message)s',
                   filemode='w')

# Log start of experiment 
logging.info(f"Output Log File: experiment_1.log")
logging.info(f"Date: {datetime.now()}")

In [46]:
%run attack_graph_MARA.ipynb
#%run ctr-core_1.ipynb

In [47]:
# Cell 4 - Define parameters and random_steps function
# Define lambda parameter for Poisson distribution
attack_rate_list = [2]  
defense_rate_list = [0]

def random_steps(route, attack_rate=None, defense_rate=None):
    """Generate probability distribution for number of steps attacker can take"""
    x = np.arange(len(route))
    pdf_d = np.power(attack_rate, x) * np.exp(-attack_rate) / factorial(x)
    pdf_d = pdf_d / np.sum(pdf_d)
    return pdf_d

In [48]:
# Import and run the main Cut-The-Rope implementation
%run ctr-core_1.ipynb

Entry node: 1
Routes: [[1, 5]]
V: [1, 5]
as1 (defender actions): []
as2 (attacker paths): [[1, 5]]
target_list: [5]
adv_list: []
m: 1, n: 0
Output Log File: experiment_1.log
Date: 2024-12-05 17:16:27.543341
Output Log File: experiment_1.log
Date: 2024-12-05 17:17:10.904357
Output Log File: experiment_1.log
Date: 2024-12-05 17:27:10.603523
Output Log File: experiment_1.log
Date: 2024-12-05 17:35:03.269123

++++++++++++++++++++++++++++++++
attack rate = 2, defense rate = 0

++++++++++++++++++++++++++++++++
attack rate = 2, defense rate = 0
Output Log File: experiment_1.log
Date: 2024-12-05 18:24:28.361668

++++++++++++++++++++++++++++++++
attack rate = 2, defense rate = 0
Output Log File: experiment_1.log
Date: 2024-12-05 18:25:37.841348

++++++++++++++++++++++++++++++++
attack rate = 2, defense rate = 0
Output Log File: experiment_1.log
Date: 2024-12-05 18:32:42.610659



In [24]:
# Read and display contents of log file
with open('experiment_1.log', 'r') as f:
    print(f.read())

Output Log File: experiment_1.log
Date: 2024-12-05 17:16:27.543341
Output Log File: experiment_1.log
Date: 2024-12-05 17:17:10.904357
Output Log File: experiment_1.log
Date: 2024-12-05 17:27:10.603523

